<a href="https://colab.research.google.com/github/KGuzikowski/AlgTrading/blob/master/Trader.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup


In [2]:
url = 'https://anaconda.org/conda-forge/libta-lib/0.4.0/download/linux-64/libta-lib-0.4.0-h516909a_0.tar.bz2'
!curl -L $url | tar xj -C /usr/lib/x86_64-linux-gnu/ lib --strip-components=1
url = 'https://anaconda.org/conda-forge/ta-lib/0.4.19/download/linux-64/ta-lib-0.4.19-py37ha21ca33_2.tar.bz2'
!curl -L $url | tar xj -C /usr/local/lib/python3.7/dist-packages/ lib/python3.7/site-packages/talib --strip-components=3

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  3729    0  3729    0     0   3421      0 --:--:--  0:00:01 --:--:--  3421
100  503k  100  503k    0     0   177k      0  0:00:02  0:00:02 --:--:--  374k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  3733    0  3733    0     0   3431      0 --:--:--  0:00:01 --:--:--  3431
100  406k  100  406k    0     0   145k      0  0:00:02  0:00:02 --:--:--  353k


In [3]:
!pip install python-binance
!pip install binance-client

     |████████████████████████████████| 63 kB 1.5 MB/s 
     |████████████████████████████████| 103 kB 25.6 MB/s 
     |████████████████████████████████| 214 kB 44.4 MB/s 
     |████████████████████████████████| 288 kB 40.9 MB/s 
     |████████████████████████████████| 1.3 MB 43.0 MB/s 
     |████████████████████████████████| 160 kB 59.7 MB/s 
     |████████████████████████████████| 271 kB 57.0 MB/s 


In [4]:
%matplotlib inline

import binance.client
import pandas as pd
import numpy as np
import time
import os
import requests
from pprint import pprint
import subprocess
import datetime as dt
import json
import pandas as pd
import talib

import matplotlib.pyplot as plt
import seaborn as sns

# from xgboost import XGBClassifier
from tqdm import tqdm

# Binance data fetch

In [5]:
class BinanceClient():
    column_names = [
        'open_time',
        'open',
        'high',
        'low',
        'close',
        'vol',
        'close_time',
        'quote_asset_volume',
        'number_of_trades',
        'taker_buy_base_asset_volume',
        'taker_buy_quote_asset_volume',
        'ignore'
    ]

    def __init__(self, api_key, api_secret, interval):
        self.client = binance.client.Client(api_key, api_secret)
        self.INTERVAL = interval

    def polish_kline(self, raw_kline):
        kline_d =  {
            'open_time': raw_kline[0],
            'open': raw_kline[1],
            'close': raw_kline[2],
            'low': raw_kline[3],
            'high': raw_kline[4],
            'vol': raw_kline[5]
        }
        kline_d['open_datetime'] = pd.to_datetime(kline_d['open_time'], unit='ms')
        return kline_d

    def historical_klines_generator(self, instrument, interval, start_dt, end_dt=None):
        for kline in self.client.get_historical_klines_generator(instrument, interval, str(start_dt), str(end_dt) if end_dt else None):
            yield self.polish_kline(kline)

In [6]:
api_key = "vT0XoiF4FbrpHbH04ZEew6A9jOORd79J4RdmId0CTT6wwX97EuY9hPCBcp9F904f"
api_secret = "wcpF4Ne1tNGbq5vPx2oaXrpbE2JnNXhn6tKV6I32uKNdBMuEazRfDPiMJkIVqVr8"

In [ ]:
client = BinanceClient(api_key, api_secret, binance.client.Client.KLINE_INTERVAL_5MINUTE)

start_date = dt.datetime(2021,10,7,10,0,0)
instrument = "BTCUSDT"

kline_df = pd.DataFrame(columns=["open","high","low","close","vol"])
for kline in client.historical_klines_generator(instrument, client.INTERVAL, start_date):
    kline_df.loc[kline['open_datetime']] = (kline["open"], kline["high"], kline["low"], kline["close"], kline["vol"])


kline_df.open = kline_df.open.astype(float)
kline_df.close = kline_df.close.astype(float)
kline_df.low = kline_df.low.astype(float)
kline_df.high = kline_df.high.astype(float)
kline_df.vol = kline_df.vol.astype(float)

In [ ]:
kline_df

# Indicators

## Indicator base class

In [ ]:
class Indicator:
    prev_states = []

    def __init__(self, timeperiod):
        self._timeperiod = timeperiod

    def __function(self):
        raise NotImplementedError()

    def __call__(self, df):
        """
        return:
        1 -> sell
        0 -> do nothing
        -1 -> buy
        """
        return self._function(df)

## Indicators' definitions

### RSI

In [ ]:
class RSI(Indicator):
    def __init__(self, timeperiod, lower, upper):
        super().__init__(timeperiod)
        self._lower = lower
        self._upper = upper

    def _function(self, df):
        rsi = talib.RSI(df.close.tail(self._timeperiod + 1), timeperiod=self._timeperiod).iloc[-1]
        
        if rsi >= self._upper:
            return 1
        elif rsi <= self._lower:
            return -1
        else:
            return 0


In [ ]:
rsi_20 = RSI(timeperiod=20, lower=35, upper=70)
print(rsi_20(kline_df))
rsi_100 = RSI(timeperiod=100, lower=40, upper=60)
print(rsi_100(kline_df))
rsi_700 = RSI(timeperiod=700, lower=49.5, upper=55)
print(rsi_700(kline_df))

0
-1
-1


### Bollinger Bands

In [ ]:
class BollingerBands(Indicator):
    def __init__(self, timeperiod, lower=2, upper=3):
        super().__init__(timeperiod)
        self._lower = lower
        self._upper = upper

    def _function(self, df):
        lower, middle, upper = talib.BBANDS(
            df.close.tail(self._timeperiod + 1),
            timeperiod=self._timeperiod,
            nbdevup=self._upper,
            nbdevdn=self._lower,
            matype=0,
        )
        closing =  df.close.iloc[-1]

        if closing >= upper.iloc[-1]:
            return 1
        elif closing <= lower.iloc[-1]:
            return -1
        else:
            return 0

In [ ]:
bb = BollingerBands(timeperiod=700, lower=2, upper=3)
bb(kline_df)

-1

### EMA small cross EMA big

In [ ]:
class EMACross(Indicator):
    """
    If small EMA crosses big EMA from bottom -> buy
    If small EMA crosses big EMA from top -> sell
    """
    def __init__(self, timeperiod_S=100, timeperiod_B=500):
        self._timeperiod_S = timeperiod_S
        self._timeperiod_B = timeperiod_B

    def _function(self, df):
        ma_S = talib.EMA(df.close.tail(self._timeperiod_S + 1), timeperiod=self._timeperiod_S)
        ma_B = talib.EMA(df.close.tail(self._timeperiod_B + 1), timeperiod=self._timeperiod_B)

        ma_S0 = ma_S.iloc[-2]
        ma_S1 = ma_S.iloc[-1]
        ma_B0 = ma_B.iloc[-2]
        ma_B1 = ma_B.iloc[-1]

        if ma_S0 >= ma_B0 and ma_S1 <= ma_B1:
            # cross from top
            return 1
        elif ma_S0 <= ma_B0 and ma_S1 >= ma_B1:
            # cross from bottom
            return -1
        else:
            return 0

In [ ]:
ema_cross = EMACross(timeperiod_S=100, timeperiod_B=500)
ema_cross(kline_df)

0

### Normalized EMA small cross normalized EMA big

In [ ]:
class NormalizedEMACross(Indicator):
    """
    If small EMA crosses big EMA from bottom -> buy
    If small EMA crosses big EMA from top -> sell
    """
    def __init__(self, timeperiod_S=100, timeperiod_B=500):
        self._timeperiod_S = timeperiod_S
        self._timeperiod_B = timeperiod_B

    def _function(self, df):
        ma_S = talib.EMA(df.close.tail(self._timeperiod_S + 1), timeperiod=self._timeperiod_S)
        ma_S = (ma_S-ma_S.mean())/ma_S.std()
        ma_B = talib.EMA(df.close.tail(self._timeperiod_B + 1), timeperiod=self._timeperiod_B)
        ma_B = (ma_B-ma_B.mean())/ma_B.std()

        ma_S0 = ma_S.iloc[-2]
        ma_S1 = ma_S.iloc[-1]
        ma_B0 = ma_B.iloc[-2]
        ma_B1 = ma_B.iloc[-1]

        if ma_S0 >= ma_B0 and ma_S1 <= ma_B1:
            # cross from top
            return 1
        elif ma_S0 <= ma_B0 and ma_S1 >= ma_B1:
            # cross from bottom
            return -1
        else:
            return 0

In [ ]:
ema_cross = NormalizedEMACross(timeperiod_S=100, timeperiod_B=500)
ema_cross(kline_df)

0

### EMA
Check where close price crosses EMA

In [ ]:
class EMA(Indicator):
    """
    If price crosses EMA from bottom -> buy
    If price crosses EMA from top -> sell
    """
    def __init__(self, timeperiod):
        super().__init__(timeperiod)

    def _function(self, df):
        ma = talib.EMA(df.close.tail(self._timeperiod + 1), timeperiod=self._timeperiod)

        ma0 = ma.iloc[-2]
        ma1 = ma.iloc[-1]
        close0 = df.close.iloc[-2]
        close1 = df.close.iloc[-1]

        if ma0 >= close0 and ma1 <= close1:
            # cross from top
            return 1
        elif ma0 <= close0 and ma1 >= close1:
            # cross from bottom
            return -1
        else:
            return 0

In [ ]:
ema_100 = EMA(timeperiod=100)
print(ema_100(kline_df))
ema_500 = EMA(timeperiod=500)
print(ema_500(kline_df))

0
0


### MACD